<img align="left" src="imgs/logo.jpg" width="50px" style="margin-right:10px">

# Snorkel Workshop: Extracting Spouse Relations <br> from the News
## Part 2: Writing  Labeling Functions

In Snorkel, our primary interface through which we provide training signal to the end extraction model we are training is by writing **labeling functions (LFs)** (as opposed to hand-labeling massive training sets).  We'll go through some examples for our spouse classification task below.

A labeling function isn't anything special. It's just a Python function that accepts a candidate, or a row of the DataFrame, as the input argument and returns `1` if it says the pair of persons in the candidate were married at some point,  `-1` if the pair of persons in the candidate were never married, and `0` if it doesn't know how to vote and abstains. In practice, many labeling functions are unipolar: it labels only `1`s and `0`s, or it labels only `-1`s and `0`s.

Recall that our goal is to ultimately train a high-performance classification model that predicts which of our candidates are true spouse relations.  It turns out that we can do this by writing potentially low-quality labeling functions!

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import re
import sys
import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##  I. Background

### Preprocessing the Database

In a real application, there is a lot of data preparation, parsing, and database loading that needs to be completed before we dive into writing labeling functions. Here we've pre-generated candidates in a pandas DataFrame object per split (train,dev,test).

###  Using a _Development Set_ of Human-labeled Data

In our setting, we will use the phrase _development set_ to refer to a set of examples (here, a subset of our training set) which we label by hand and use to help us develop and refine labeling functions.  Unlike the _test set_, which we do not look at and use for final evaluation, we can inspect the development set while writing labeling functions. This is a list of `{-1,1}` labels.

In [8]:
import pickle

with open('token_dev_data.pkl', 'rb') as f:
    dev_data = pickle.load(f)
    dev_labels = pickle.load(f)
    
with open('token_train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)
    train_labels = pickle.load(f)

### Labeling Function Metrics

#### Coverage
One simple metric we can compute quickly is our _coverage_, the number of candidates labeled by our LF, on our training set (or any other set).

#### Precision / Recall / F1
If we have gold labeled data, we can also compute standard precision, recall, and F1 metrics for the output of a single labeling function. These metrics are computed over 4 _error buckets_: _True Positives_ (tp), _False Positives_ (fp), _True Negatives_ (tn), and _False Negatives_ (fn).

\begin{equation*}
precision = \frac{tp}{(tp + fp)}
\end{equation*}

\begin{equation*}
recall = \frac{tp}{(tp + fn)}
\end{equation*}

\begin{equation*}
F1 = 2 \cdot \frac{ (precision \cdot recall)}{(precision + recall)}
\end{equation*}

# II. Labeling Functions

## A. Pattern Matching Labeling Functions

One powerful form of labeling function design is defining sets of keywords or regular expressions that, as a human labeler, you know are correlated with the true label. In the terminology of [Bayesian inference](https://en.wikipedia.org/wiki/Statistical_inference#Bayesian_inference), this can be thought of as defining a [_prior_](https://en.wikipedia.org/wiki/Prior_probability) over your word features. 

For example, we could define a dictionary of terms that occur between person names in a candidate. One simple dictionary of terms indicating a true relation could be:
    
    spouses = {'husband', 'wife'}
 
We can then write a labeling function that checks for a match with these terms in the text that occurs between person names. To access the text between the person mentions, we can use the `get_between_tokens` preprocessors from the previous notebook.

    @labeling_function(resources=dict(spouses=spouses), preprocessors=[get_between_tokens])
    def LF_husband_wife(x: DataPoint, spouses: List[str]) -> int:
        return 1 if len(spouses.intersection(set(x.between_tokens))) > 0 else 0
        
The idea is that we can easily create dictionaries that encode themes or categories descibing all kinds of relationships between 2 people and then use these objects to _weakly supervise_ our classification task.

    other_relationship = {'boyfriend', 'girlfriend'}
    
**IMPORTANT** Good labeling functions manage a trade-off between high coverage and high precision. When constructing your dictionaries, think about building larger, noiser sets of terms instead of relying on 1 or 2 keywords. Sometimes a single word can be very predictive (e.g., `ex-wife`) but it's almost always better to define something more general, such as a regular expression pattern capturing _any_ string with the `ex-` prefix. 

In [9]:
from typing import List

from snorkel.labeling.apply import PandasLFApplier
from snorkel.labeling.lf import labeling_function
from snorkel.types import DataPoint

from spouse_preprocessors import get_between_tokens, get_left_tokens, get_right_tokens, \
get_person_last_names, get_person_text
    

spouses = ['spouse', 'wife', 'husband', 'ex-wife', 'ex-husband']
@labeling_function(resources=dict(spouses=spouses), preprocessors=[get_between_tokens])
def LF_husband_wife(x: DataPoint, spouses: List[str]) -> int:
    return 1 if len(set(spouses).intersection(set(x.between_tokens))) > 0 else 0

@labeling_function(resources=dict(spouses=spouses), preprocessors=[get_left_tokens])
def LF_husband_wife_left_window(x: DataPoint, spouses: List[str]) -> int:
    if len(set(spouses).intersection(set(x.person1_left_tokens))) > 0:
        return 1
    elif len(set(spouses).intersection(set(x.person2_left_tokens))) > 0:
        return 1
    else:
        return 0

@labeling_function()
def LF_same_last_name(x: DataPoint) -> int:
    p1_ln, p2_ln = get_person_last_names(x)
    
    if p1_ln and p2_ln and p1_ln == p2_ln:
        if ' '.join(p1_ln) != ' '.join(p2_ln):
            return 1
    return 0

@labeling_function(preprocessors=[get_between_tokens, get_right_tokens])
def LF_and_married(x: DataPoint) -> int:
    return 1 if 'and' in x.between_tokens and 'married' in x.person2_right_tokens else 0    


family = ['father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin']
family = family+[f + '-in-law' for f in family]

@labeling_function(resources=dict(family=family), preprocessors=[get_between_tokens])
def LF_familial_relationship(x: DataPoint, family: List[str]) -> int:
    return 1 if len(set(family).intersection(set(x.between_tokens))) > 0 else 0  


@labeling_function(resources=dict(family=family), preprocessors=[get_left_tokens])
def LF_family_left_window(x: DataPoint, family: List[str]) -> int:
    if len(set(family).intersection(set(x.person1_left_tokens))) > 0:
        return -1
    elif len(set(family).intersection(set(x.person2_left_tokens))) > 0:
        return -1
    else:
        return 0

other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}
@labeling_function(resources=dict(other=other), preprocessors=[get_between_tokens])
def LF_other_relationship(x: DataPoint, other: List[str]) -> int:
    return -1 if len(set(other).intersection(set(x.between_tokens))) > 0 else 0

In [10]:
applier = PandasLFApplier([LF_husband_wife,
                           LF_husband_wife_left_window,
                           LF_same_last_name,
                           LF_and_married, 
                           LF_familial_relationship,
                           LF_family_left_window,
                           LF_other_relationship])
L = applier.apply(dev_data)



  0%|          | 0/2811 [00:00<?, ?it/s]

  0%|          | 14/2811 [00:00<00:21, 130.63it/s]

  1%|          | 27/2811 [00:00<00:21, 128.23it/s]

  1%|▏         | 40/2811 [00:00<00:21, 126.69it/s]

  2%|▏         | 53/2811 [00:00<00:21, 126.71it/s]

  2%|▏         | 66/2811 [00:00<00:21, 127.27it/s]

  3%|▎         | 79/2811 [00:00<00:21, 127.58it/s]

  3%|▎         | 92/2811 [00:00<00:21, 127.73it/s]

  4%|▍         | 106/2811 [00:00<00:21, 128.57it/s]

  4%|▍         | 119/2811 [00:00<00:20, 128.31it/s]

  5%|▍         | 132/2811 [00:01<00:20, 127.82it/s]

  5%|▌         | 145/2811 [00:01<00:20, 127.59it/s]

  6%|▌         | 158/2811 [00:01<00:20, 127.71it/s]

  6%|▌         | 171/2811 [00:01<00:20, 126.36it/s]

  7%|▋         | 184/2811 [00:01<00:20, 125.53it/s]

  7%|▋         | 197/2811 [00:01<00:20, 125.88it/s]

  7%|▋         | 210/2811 [00:01<00:20, 126.66it/s]

  8%|▊         | 223/2811 [00:01<00:20, 127.59it/s]

  8%|▊         | 236/2811 [00:01<00:20, 128.09it/s]

  9%|▉   

 63%|██████▎   | 1764/2811 [00:14<00:08, 119.27it/s]

 63%|██████▎   | 1776/2811 [00:14<00:08, 118.17it/s]

 64%|██████▎   | 1789/2811 [00:14<00:08, 120.30it/s]

 64%|██████▍   | 1802/2811 [00:14<00:08, 119.30it/s]

 65%|██████▍   | 1814/2811 [00:14<00:08, 119.15it/s]

 65%|██████▍   | 1827/2811 [00:14<00:08, 120.87it/s]

 65%|██████▌   | 1840/2811 [00:15<00:07, 122.84it/s]

 66%|██████▌   | 1853/2811 [00:15<00:07, 123.70it/s]

 66%|██████▋   | 1866/2811 [00:15<00:07, 124.99it/s]

 67%|██████▋   | 1879/2811 [00:15<00:07, 125.29it/s]

 67%|██████▋   | 1892/2811 [00:15<00:07, 125.03it/s]

 68%|██████▊   | 1905/2811 [00:15<00:07, 125.09it/s]

 68%|██████▊   | 1918/2811 [00:15<00:07, 125.92it/s]

 69%|██████▊   | 1931/2811 [00:15<00:06, 126.79it/s]

 69%|██████▉   | 1944/2811 [00:15<00:06, 126.42it/s]

 70%|██████▉   | 1957/2811 [00:16<00:06, 124.86it/s]

 70%|███████   | 1970/2811 [00:16<00:06, 124.58it/s]

 71%|███████   | 1983/2811 [00:16<00:06, 124.60it/s]

 71%|███████   | 1996/2811 [

**Viewing Performance Metrics**
If we have gold labeled data, we can evaluate formal metrics. It's useful to view specific error metrics for a particular LF. Below, we'll compute our empirical scores using human-labeled development set data and then look at performance metrics for `LF_husband_wife` LF. Notice the precision and recall are both only around 35\%!

In [11]:
from snorkel.model.metrics import coverage_score, f1_score, precision_score, recall_score

print("LF_husband_wife coverage: \t", coverage_score(dev_labels,L[:,0]))
print("LF_husband_wife F1 score:  \t", f1_score(dev_labels,L[:,0]))
print("LF_husband_wife precision:  \t", precision_score(dev_labels,L[:,0]))
print("LF_husband_wife F1 recall:  \t", recall_score(dev_labels,L[:,0]))

LF_husband_wife coverage: 	 0.08964781216648879
LF_husband_wife F1 score:  	 0.4208144796380091
LF_husband_wife precision:  	 0.36904761904761907
LF_husband_wife F1 recall:  	 0.48947368421052634


## B. Distant Supervision Labeling Functions

In addition to using factories that encode pattern matching heuristics, we can also write labeling functions that _distantly supervise_ examples. Here, we'll load in a list of known spouse pairs and check to see if the pair of persons in a candidate matches one of these.

**DBpedia**
http://wiki.dbpedia.org/
Out database of known spouses comes from DBpedia, which is a community-driven resource similar to Wikipedia but for curating structured data. We'll use a preprocessed snapshot as our knowledge base for all labeling function development.

We can look at some of the example entries from DBPedia and use them in a simple distant supervision labeling function.

Make sure `dbpedia.pkl` is in the `tutorials/workshop/` directory. 

In [13]:
import pickle 

with open('dbpedia.pkl', 'rb') as f:
     known_spouses = pickle.load(f)
        
list(known_spouses)[0:5]

[('Alex Weed', 'Fiona Gubelmann'),
 ('Henry Ephron', 'Phoebe Ephron'),
 ('John Reed', 'Sunday Reed'),
 ('Agnes of the Palatinate', 'Otto II Wittelsbach Duke of Bavaria'),
 ('Alexandrine of Mecklenburg-Schwerin', 'Christian X')]

In [17]:
@labeling_function(resources=dict(known_spouses=known_spouses))
def LF_distant_supervision(x: DataPoint, known_spouses: List[str]) -> int:
    p1, p2 = get_person_text(x)
    return 1 if (p1, p2) in known_spouses or (p2, p1) in known_spouses else 0


# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None 

# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])

@labeling_function(resources=dict(last_names=last_names))
def LF_distant_supervision_last_names(x: DataPoint, last_names: List[str]) -> int:
    p1_ln, p2_ln = get_person_last_names(x)
    
    return 1 if (p1_ln != p2_ln) and ((p1_ln, p2_ln) in last_names or (p2_ln, p1_ln) in last_names) else 0 

In [18]:
applier = PandasLFApplier([LF_husband_wife,
                           LF_husband_wife_left_window,
                           LF_same_last_name,
                           LF_and_married, 
                           LF_familial_relationship,
                           LF_family_left_window,
                           LF_other_relationship,
                           LF_distant_supervision,
                           LF_distant_supervision_last_names])

In [19]:
dev_L = applier.apply(dev_data)
with open('dev_L.pkl', 'wb') as f:
    pickle.dump(dev_L, f)
    
train_L = applier.apply(train_data)
with open('train_L.pkl', 'wb') as f:
    pickle.dump(train_L, f)



  0%|          | 0/2811 [00:00<?, ?it/s]

  0%|          | 13/2811 [00:00<00:22, 123.10it/s]

  1%|          | 26/2811 [00:00<00:22, 122.95it/s]

  1%|▏         | 39/2811 [00:00<00:22, 123.85it/s]

  2%|▏         | 52/2811 [00:00<00:22, 123.14it/s]

  2%|▏         | 64/2811 [00:00<00:22, 119.46it/s]

  3%|▎         | 77/2811 [00:00<00:22, 120.43it/s]

  3%|▎         | 90/2811 [00:00<00:22, 121.35it/s]

  4%|▎         | 103/2811 [00:00<00:22, 122.46it/s]

  4%|▍         | 116/2811 [00:00<00:21, 123.56it/s]

  5%|▍         | 129/2811 [00:01<00:21, 123.94it/s]

  5%|▌         | 142/2811 [00:01<00:21, 124.13it/s]

  6%|▌         | 155/2811 [00:01<00:21, 121.94it/s]

  6%|▌         | 168/2811 [00:01<00:21, 121.33it/s]

  6%|▋         | 181/2811 [00:01<00:21, 121.54it/s]

  7%|▋         | 194/2811 [00:01<00:21, 121.65it/s]

  7%|▋         | 207/2811 [00:01<00:21, 121.03it/s]

  8%|▊         | 220/2811 [00:01<00:21, 120.98it/s]

  8%|▊         | 233/2811 [00:01<00:21, 122.57it/s]

  9%|▉   

 63%|██████▎   | 1781/2811 [00:14<00:08, 117.15it/s]

 64%|██████▍   | 1793/2811 [00:14<00:08, 117.29it/s]

 64%|██████▍   | 1806/2811 [00:14<00:08, 119.18it/s]

 65%|██████▍   | 1819/2811 [00:15<00:08, 120.30it/s]

 65%|██████▌   | 1832/2811 [00:15<00:08, 120.87it/s]

 66%|██████▌   | 1845/2811 [00:15<00:07, 121.02it/s]

 66%|██████▌   | 1858/2811 [00:15<00:07, 121.42it/s]

 67%|██████▋   | 1871/2811 [00:15<00:07, 122.08it/s]

 67%|██████▋   | 1884/2811 [00:15<00:07, 122.38it/s]

 67%|██████▋   | 1897/2811 [00:15<00:07, 119.59it/s]

 68%|██████▊   | 1909/2811 [00:15<00:07, 119.69it/s]

 68%|██████▊   | 1922/2811 [00:15<00:07, 121.54it/s]

 69%|██████▉   | 1935/2811 [00:15<00:07, 123.04it/s]

 69%|██████▉   | 1948/2811 [00:16<00:07, 123.23it/s]

 70%|██████▉   | 1961/2811 [00:16<00:06, 122.20it/s]

 70%|███████   | 1974/2811 [00:16<00:06, 122.78it/s]

 71%|███████   | 1987/2811 [00:16<00:06, 121.24it/s]

 71%|███████   | 2000/2811 [00:16<00:06, 121.71it/s]

 72%|███████▏  | 2013/2811 [

  3%|▎         | 659/22254 [00:05<02:59, 120.24it/s]

  3%|▎         | 672/22254 [00:05<02:58, 120.62it/s]

  3%|▎         | 685/22254 [00:06<03:04, 116.87it/s]

  3%|▎         | 698/22254 [00:06<03:02, 118.13it/s]

  3%|▎         | 710/22254 [00:06<03:06, 115.62it/s]

  3%|▎         | 723/22254 [00:06<03:02, 118.18it/s]

  3%|▎         | 735/22254 [00:06<03:02, 117.95it/s]

  3%|▎         | 748/22254 [00:06<02:59, 119.61it/s]

  3%|▎         | 760/22254 [00:06<03:03, 117.28it/s]

  3%|▎         | 773/22254 [00:06<03:00, 119.32it/s]

  4%|▎         | 786/22254 [00:06<02:57, 121.02it/s]

  4%|▎         | 799/22254 [00:06<02:58, 120.17it/s]

  4%|▎         | 812/22254 [00:07<02:58, 119.97it/s]

  4%|▎         | 825/22254 [00:07<02:59, 119.71it/s]

  4%|▍         | 838/22254 [00:07<02:57, 120.44it/s]

  4%|▍         | 851/22254 [00:07<02:56, 121.03it/s]

  4%|▍         | 864/22254 [00:07<02:55, 122.20it/s]

  4%|▍         | 877/22254 [00:07<02:59, 119.05it/s]

  4%|▍         | 890/22254 [

 11%|█         | 2398/22254 [00:19<02:39, 124.32it/s]

 11%|█         | 2411/22254 [00:20<02:41, 123.11it/s]

 11%|█         | 2424/22254 [00:20<02:40, 123.79it/s]

 11%|█         | 2437/22254 [00:20<02:40, 123.38it/s]

 11%|█         | 2450/22254 [00:20<02:42, 121.66it/s]

 11%|█         | 2463/22254 [00:20<02:41, 122.24it/s]

 11%|█         | 2476/22254 [00:20<02:43, 121.03it/s]

 11%|█         | 2489/22254 [00:20<02:41, 122.60it/s]

 11%|█         | 2502/22254 [00:20<02:39, 123.55it/s]

 11%|█▏        | 2515/22254 [00:20<02:40, 123.32it/s]

 11%|█▏        | 2528/22254 [00:21<02:40, 122.83it/s]

 11%|█▏        | 2541/22254 [00:21<02:40, 123.13it/s]

 11%|█▏        | 2554/22254 [00:21<02:39, 123.39it/s]

 12%|█▏        | 2567/22254 [00:21<02:39, 123.18it/s]

 12%|█▏        | 2580/22254 [00:21<02:38, 123.74it/s]

 12%|█▏        | 2593/22254 [00:21<02:40, 122.84it/s]

 12%|█▏        | 2606/22254 [00:21<02:39, 122.86it/s]

 12%|█▏        | 2619/22254 [00:21<02:41, 121.50it/s]

 12%|█▏   

 19%|█▊        | 4138/22254 [00:34<02:28, 121.65it/s]

 19%|█▊        | 4151/22254 [00:34<02:27, 122.43it/s]

 19%|█▊        | 4164/22254 [00:34<02:28, 122.05it/s]

 19%|█▉        | 4177/22254 [00:34<02:34, 116.77it/s]

 19%|█▉        | 4189/22254 [00:34<02:40, 112.36it/s]

 19%|█▉        | 4201/22254 [00:34<02:40, 112.67it/s]

 19%|█▉        | 4214/22254 [00:34<02:36, 115.04it/s]

 19%|█▉        | 4226/22254 [00:34<02:34, 116.42it/s]

 19%|█▉        | 4238/22254 [00:35<02:35, 115.89it/s]

 19%|█▉        | 4251/22254 [00:35<02:32, 117.83it/s]

 19%|█▉        | 4264/22254 [00:35<02:31, 118.72it/s]

 19%|█▉        | 4277/22254 [00:35<02:30, 119.62it/s]

 19%|█▉        | 4290/22254 [00:35<02:28, 121.02it/s]

 19%|█▉        | 4303/22254 [00:35<02:27, 122.02it/s]

 19%|█▉        | 4316/22254 [00:35<02:27, 121.99it/s]

 19%|█▉        | 4329/22254 [00:35<02:26, 122.26it/s]

 20%|█▉        | 4342/22254 [00:35<02:25, 122.71it/s]

 20%|█▉        | 4355/22254 [00:36<02:28, 120.58it/s]

 20%|█▉   

...

 65%|██████▍   | 14433/22254 [01:58<01:06, 117.64it/s]

 65%|██████▍   | 14445/22254 [01:58<01:09, 111.68it/s]

 65%|██████▍   | 14457/22254 [01:58<01:10, 111.34it/s]

 65%|██████▌   | 14469/22254 [01:58<01:08, 113.39it/s]

 65%|██████▌   | 14482/22254 [01:59<01:06, 116.32it/s]

 65%|██████▌   | 14495/22254 [01:59<01:05, 119.34it/s]

 65%|██████▌   | 14508/22254 [01:59<01:05, 119.05it/s]

 65%|██████▌   | 14520/22254 [01:59<01:05, 118.66it/s]

 65%|██████▌   | 14533/22254 [01:59<01:04, 120.21it/s]

 65%|██████▌   | 14546/22254 [01:59<01:03, 120.83it/s]

 65%|██████▌   | 14559/22254 [01:59<01:03, 121.48it/s]

 65%|██████▌   | 14572/22254 [01:59<01:03, 121.89it/s]

 66%|██████▌   | 14585/22254 [01:59<01:03, 120.87it/s]

 66%|██████▌   | 14598/22254 [01:59<01:03, 120.60it/s]

 66%|██████▌   | 14611/22254 [02:00<01:03, 121.29it/s]

 66%|██████▌   | 14624/22254 [02:00<01:02, 121.52it/s]

 66%|██████▌   | 14637/22254 [02:00<01:01, 123.67it/s]

 66%|██████▌   | 14650/22254 [02:00<01:03, 120.6

 72%|███████▏  | 16117/22254 [02:12<00:48, 127.82it/s]

 72%|███████▏  | 16130/22254 [02:12<00:47, 127.67it/s]

 73%|███████▎  | 16143/22254 [02:12<00:48, 126.15it/s]

 73%|███████▎  | 16156/22254 [02:13<00:49, 124.37it/s]

 73%|███████▎  | 16169/22254 [02:13<00:48, 124.57it/s]

 73%|███████▎  | 16182/22254 [02:13<00:49, 121.78it/s]

 73%|███████▎  | 16195/22254 [02:13<00:50, 119.74it/s]

 73%|███████▎  | 16207/22254 [02:13<00:53, 113.57it/s]

 73%|███████▎  | 16219/22254 [02:13<00:53, 113.81it/s]

 73%|███████▎  | 16232/22254 [02:13<00:51, 115.86it/s]

 73%|███████▎  | 16244/22254 [02:13<00:52, 115.02it/s]

 73%|███████▎  | 16256/22254 [02:13<00:52, 113.70it/s]

 73%|███████▎  | 16269/22254 [02:14<00:51, 116.08it/s]

 73%|███████▎  | 16282/22254 [02:14<00:50, 118.44it/s]

 73%|███████▎  | 16295/22254 [02:14<00:49, 121.02it/s]

 73%|███████▎  | 16308/22254 [02:14<00:51, 116.52it/s]

 73%|███████▎  | 16320/22254 [02:14<00:56, 104.71it/s]

 73%|███████▎  | 16331/22254 [02:14<01:01, 97.00

 79%|███████▉  | 17686/22254 [02:27<00:38, 117.31it/s]

 80%|███████▉  | 17699/22254 [02:27<00:38, 119.58it/s]

 80%|███████▉  | 17712/22254 [02:27<00:37, 120.58it/s]

 80%|███████▉  | 17725/22254 [02:27<00:37, 121.82it/s]

 80%|███████▉  | 17738/22254 [02:27<00:37, 121.30it/s]

 80%|███████▉  | 17751/22254 [02:27<00:37, 121.35it/s]

 80%|███████▉  | 17764/22254 [02:27<00:36, 122.75it/s]

 80%|███████▉  | 17777/22254 [02:27<00:38, 116.50it/s]

 80%|███████▉  | 17790/22254 [02:27<00:37, 119.01it/s]

 80%|███████▉  | 17803/22254 [02:28<00:37, 120.23it/s]

 80%|████████  | 17816/22254 [02:28<00:36, 121.63it/s]

 80%|████████  | 17829/22254 [02:28<00:36, 122.61it/s]

 80%|████████  | 17842/22254 [02:28<00:35, 123.37it/s]

 80%|████████  | 17855/22254 [02:28<00:35, 123.97it/s]

 80%|████████  | 17868/22254 [02:28<00:35, 124.01it/s]

 80%|████████  | 17881/22254 [02:28<00:35, 124.07it/s]

 80%|████████  | 17894/22254 [02:28<00:35, 124.29it/s]

 80%|████████  | 17907/22254 [02:28<00:34, 124.7

 87%|████████▋ | 19376/22254 [02:41<00:24, 117.64it/s]

 87%|████████▋ | 19389/22254 [02:41<00:24, 118.54it/s]

 87%|████████▋ | 19401/22254 [02:41<00:24, 118.52it/s]

 87%|████████▋ | 19414/22254 [02:41<00:23, 119.76it/s]

 87%|████████▋ | 19427/22254 [02:41<00:23, 120.65it/s]

 87%|████████▋ | 19440/22254 [02:41<00:23, 121.02it/s]

 87%|████████▋ | 19453/22254 [02:41<00:22, 122.10it/s]

 87%|████████▋ | 19466/22254 [02:41<00:22, 122.11it/s]

 88%|████████▊ | 19479/22254 [02:42<00:22, 121.21it/s]

 88%|████████▊ | 19492/22254 [02:42<00:22, 121.26it/s]

 88%|████████▊ | 19505/22254 [02:42<00:22, 121.35it/s]

 88%|████████▊ | 19518/22254 [02:42<00:22, 121.26it/s]

 88%|████████▊ | 19531/22254 [02:42<00:22, 122.26it/s]

 88%|████████▊ | 19544/22254 [02:42<00:22, 123.16it/s]

 88%|████████▊ | 19557/22254 [02:42<00:22, 122.57it/s]

 88%|████████▊ | 19570/22254 [02:42<00:22, 119.85it/s]

 88%|████████▊ | 19583/22254 [02:42<00:22, 120.39it/s]

 88%|████████▊ | 19596/22254 [02:42<00:22, 120.4

 95%|█████████▍| 21091/22254 [02:54<00:09, 127.66it/s]

 95%|█████████▍| 21104/22254 [02:55<00:09, 127.44it/s]

 95%|█████████▍| 21117/22254 [02:55<00:08, 128.03it/s]

 95%|█████████▍| 21130/22254 [02:55<00:08, 128.13it/s]

 95%|█████████▌| 21143/22254 [02:55<00:08, 128.25it/s]

 95%|█████████▌| 21156/22254 [02:55<00:08, 128.05it/s]

 95%|█████████▌| 21169/22254 [02:55<00:08, 128.15it/s]

 95%|█████████▌| 21182/22254 [02:55<00:08, 127.79it/s]

 95%|█████████▌| 21195/22254 [02:55<00:08, 128.32it/s]

 95%|█████████▌| 21208/22254 [02:55<00:08, 126.72it/s]

 95%|█████████▌| 21221/22254 [02:55<00:08, 126.93it/s]

 95%|█████████▌| 21234/22254 [02:56<00:07, 127.66it/s]

 95%|█████████▌| 21247/22254 [02:56<00:07, 126.83it/s]

 96%|█████████▌| 21260/22254 [02:56<00:07, 126.65it/s]

 96%|█████████▌| 21273/22254 [02:56<00:07, 127.13it/s]

 96%|█████████▌| 21287/22254 [02:56<00:07, 128.00it/s]

 96%|█████████▌| 21300/22254 [02:56<00:07, 127.90it/s]

 96%|█████████▌| 21313/22254 [02:56<00:07, 127.5

## C. Writing Custom Labeling Functions

The strength of LFs is that you can write any arbitrary function and use it to supervise a classification task. This approach can combine many of the same strategies discussed above or encode other information. 

For example, we observe that when mentions of person names occur far apart in a sentence, this is a good indicator that the candidate's label is False. You can write a labeling function that uses preprocessors `get_text_between` or `get_between_tokens` to write such an LF!

In [11]:
@labeling_function()
def LF_new(x: DataPoint) -> int:
    return 0

applier = PandasLFApplier([LF_husband_wife,
                           LF_husband_wife_left_window,
                           LF_same_last_name,
                           LF_and_married, 
                           LF_familial_relationship,
                           LF_family_left_window,
                           LF_other_relationship,
                           LF_distant_supervision,
                           LF_distant_supervision_last_names,
                           LF_new])

In [10]:
dev_L = applier.apply(dev_data)
with open('dev_L.pkl', 'wb') as f:
    pickle.dump(dev_L, f)
    
train_L = applier.apply(train_data)
with open('train_L.pkl', 'wb') as f:
    pickle.dump(train_L, f)

100%|██████████| 22254/22254 [02:48<00:00, 131.97it/s]
